# El problema de las tarjetas numeradas

Problema 4 del boletín: Se tienen diez tarjetas, numeradas de 1 a 10. El problema consiste en disponer esas tarjetas en dos pilas, $P_1$ y $P_2$, de tal forma que la suma de los números de las tarjetas en $P_1$ sea lo más próxima posible a 36 y el producto de los números de las tarjetas en $P_2$ sea los más próximo posible a 360.

Para resolver el problema mediante algoritmos genéticos definimos los parámetros necesarios de la siguiente manera:
* Los genes son 0 y 1.
* Los individuos son cromosomas de longitud 10.
* Cada individuo representa la solución en la que la tarjeta $i$-ésima va a la pila $P_1$ si el gen $i$-ésimo es 0 y a la pila $P_2$ si es 1.
* La evaluación del fenotipo de cada individuo es la diferencia en valor absoluto de la suma de las tarjetas en la pila $P_1$ con respecto a 36, más la diferencia en valor absoluto del producto de las tarjetas en $P_2$ con respecto a 360.

Se trata entonces de un problema de minimización, donde la solución perfecta sería la evaluada con 0.

## Paquete DEAP

El paquete de _Python_ [DEAP](https://deap.readthedocs.io/en/master/) proporciona un marco de trabajo para la computación evolutiva, en particular los algoritmos genéticos.

Es necesario instalar el paquete deap para realizar la práctica:

In [1]:
# Sólo si la importación de los módulos que se indica a continuación falla. 
# Puede ser necesario reiniciar el kernel a continuación.
# !pip install deap

Si la instrucción anterior produce un error, probablemente sea un problema de permisos. En este caso, prueba la siguiente instrucción:

In [2]:
# !pip install --user deap

**Importante:** Para que los cambios tengan efecto, debes reiniciar el kernel (menu Kernel > Restart), o en su defecto, cerrar completamente Anaconda y volver a abrirlo. No es suficiente con cerrar y abrir el notebook. Para comprobar que todo está correcto, ejecuta la siguiente viñeta para cargar la librería. Si no recibes ningún error es que el paquete DEAP ha sido instalado correctamente.

En primer lugar, importamos los módulos necesarios:

In [3]:
import random
from deap import base, creator, tools, algorithms
import numpy

En la literatura acerca de los algoritmos genéticos podemos encontrar multitud de maneras de representar los individuos. Debido a ello, el paquete DEAP no proporciona representaciones concretas, sino que implementa un mecanismo general para declararlas. Para atenernos a la representación de un individuo como una secuencia de genes considerada en clase, declararemos una clase `individuoTarjetas` que herede del tipo _list_ de _Python_.

Todo individuo debe tener un atributo `fitness` que guarde su evaluación. Hay que tener en cuenta no obstante que el paquete DEAP trata a la optimización uniobjetivo, en la que a cada individuo se le asocia un único valor de _fitness_, como un caso particular de optimización multiobjetivo, en la que a cada individuo se le asocian varios valores de _fitness_, con un determinado peso cada uno. Es por ello que el valor que se guarde en el atributo `fitness` de cada individuo será una tupla con el valor de cada función _fitness_ multiplicado por el peso asociado.

Por tanto, para representar las evaluaciones de los individuos deberemos declarar una clase que herede de la clase `base.Fitness` (que se encargará de automatizar el producto de los valores por los pesos) y en la que establezcamos en el atributo `weights` una tupla con los valores de los pesos, como números reales. En el caso de la optimización uniobjetivo, que es la considerada en clase, trabajaríamos con tuplas de longitud 1. Debe tenerse en cuenta que los algoritmos implementados en el paquete DEAP tratan de encontrar el individuo con evaluación máxima, por lo que para un problema de maximización bastaría usar la tupla `(1.0,)` como valor para el atributo `weights`. Para un problema de minimización habría que modificar la función *fitness*, aunque para simplemente cambiarle el signo bastaría usar la tupla `(-1.0,)` como valor para el atributo `weights`.

Por último, el paquete DEAP implementa un mecanismo, a través de la función `creator.create`, para declarar una clase de objetos que herede de la clase base especificada y tenga los valores indicados para los atributos especificados. Esas clases se crearían dentro del módulo `creator`.

Declaramos la clase `criterioTarjetas` en el módulo `creator` con la siguiente expresión:

In [4]:
creator.create('criterioTarjetas', base.Fitness, weights=(-1.0,))

A continuación declaramos la clase `individuoTarjetas` en el módulo `creator`, usando como valor para el atributo `fitness` un objeto de la clase `criterioTrajetas`, con la siguiente expresión:

In [5]:
creator.create('individuoTarjetas', list, fitness = creator.criterioTarjetas)

Ahora debemos crear una _caja de herramientas_ (_toolbox_) en la que registremos todos los elementos necesarios para poder aplicar un algoritmo genético.

In [6]:
cajaTarjetas = base.Toolbox()

El método `register` de la caja de herramientas permite registrar funciones, dándole un nombre y estableciendo unos valores por defecto de los argumentos.

En primer lugar registramos una función `genTarjetas` que devuelve un 0 o un 1 de manera aleatoria.

In [7]:
cajaTarjetas.register('genTarjetas', random.randint, 0, 1)

In [8]:
random.seed(12345)  # Semilla para el mecanismo de generación de números aleatorios
for _ in range(5):
    print(cajaTarjetas.genTarjetas())

1
0
1
1
0


Para generar un individuo debemos generar 10 genes de manera aleatoria. La función `initRepeat` del módulo `tools` aplicada a los argumentos `container`, `func` y `n` guarda en el contenedor `container` los resultados obtenidos al aplicar `n` veces la función `func`. Podemos entonces registrar de la siguiente manera en la caja de herramientas una función `generaIndTarjetas` que devuelve un individuo aleatorio.

In [9]:
cajaTarjetas.register('generaIndTarjetas', tools.initRepeat, 
                      container = creator.individuoTarjetas, 
                      func = cajaTarjetas.genTarjetas, 
                      n = 10)

In [10]:
random.seed(12345)
cajaTarjetas.generaIndTarjetas()

[1, 0, 1, 1, 0, 1, 1, 0, 1, 0]

Consideraremos una población como una lista de 10 individuos. Haciendo uso de nuevo de la función `initRepeat` podemos registrar en la caja de herramientas una función `generaPobTarjetas` que devuelve una población aleatoria.

In [11]:
cajaTarjetas.register('generaPobTarjetas', tools.initRepeat,
                      container = list, 
                      func = cajaTarjetas.generaIndTarjetas, 
                      n = 10)

In [12]:
random.seed(12345)
cajaTarjetas.generaPobTarjetas()

[[1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 0],
 [1, 0, 0, 0, 1, 0, 0, 1, 1, 0],
 [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
 [1, 0, 0, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 0, 1],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 1]]

A continuación registramos la función que permite evaluar el fenotipo de cada individuo. __La función debe devolver una tupla y los algoritmos genéticos implementados en el paquete DEAP esperan que esté registrada con el nombre__ `evaluate`.

In [13]:
def fenotipoIndTarjetas(individuo):
    P1 = []
    P2 = []
    for i in range(10):
        if individuo[i] == 0:
            P1.append(i + 1)
        else:
            P2.append(i + 1)
    return (P1, P2)

def evaluaIndTarjetas(individuo):
    P1, P2 = fenotipoIndTarjetas(individuo)
    return (abs(sum(P1) - 36) + abs(numpy.prod(P2) - 360),)

cajaTarjetas.register('evaluate', evaluaIndTarjetas)

In [14]:
fenotipoIndTarjetas([1, 1, 1, 0, 0, 1, 0, 1, 0, 1])

([4, 5, 7, 9], [1, 2, 3, 6, 8, 10])

In [15]:
cajaTarjetas.evaluate([1, 1, 1, 0, 0, 1, 0, 1, 0, 1])

(2531,)

**Pregunta:** ¿Cómo podríamos convertir este problema de minimización en uno de maximización?

Una vez registradas todas las funciones relativas a los individuos, pasamos a registrar los operadores que utilizaremos en el algoritmo genético. El paquete DEAP proporciona implementaciones de muchos operadores habituales (véase https://deap.readthedocs.io/en/master/api/tools.html#operators), entre ellos el cruce en un punto (función `cxOnePoint` del módulo `tools`) y la mutación de cadenas de bits (función `mutFlipBit` del módulo `tools`). __Los algoritmos genéticos implementados en el paquete DEAP esperan que el operador de cruce esté registrado con el nombre__ `mate` __y el operador de mutación con el nombre__ `mutate`.

In [16]:
cajaTarjetas.register('mate', tools.cxOnePoint)
cajaTarjetas.register('mutate', tools.mutFlipBit, indpb = 0.1)

El parámetro `indpb` indica la probabilidad de que un gen mute. Si por ejemplo, tenemos 10 genes y `indpb=0.3`, se modificarán, en promedio, 3 genes.

In [17]:
random.seed(12345)
cajaTarjetas.mate([1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

([1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0])

In [18]:
random.seed(12345)
cajaTarjetas.mutate([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

([0, 1, 0, 0, 0, 0, 0, 0, 0, 0],)

Finalmente, como método de selección de individuos registramos en la caja de herramientas el método de selección por torneo, en el que para seleccionar un individuo se elige al azar una cierta cantidad de individuos y de entre ellos se selecciona el más apto. Este método está implementado en la función `selTournament` del módulo `tools`. __Los algoritmos genéticos implementados en el paquete DEAP esperan que el método de selección esté registrado con el nombre__ `select`.

__Nota__: la función _fitness_ que hemos definido es no negativa y, para transformar este problema de minimización en un problema de maximización, sus valores se ven luego multiplicados por el peso `-1.0` que establecimos en la clase `criterioTarjetas`. Por lo tanto, no es posible usar por esta vía el método de selección por ruleta aleatoria, ya que todos los valores con los que se trabaja finalmente son negativos. En caso de querer usar este método, habría que modificar la función _fitness_ como se explicó en el tema y usar `1.0` como peso.

In [19]:
cajaTarjetas.register('select', tools.selTournament, tournsize=3)

In [20]:
random.seed(12345)
P = cajaTarjetas.generaPobTarjetas()
cajaTarjetas.select(P, 5)

[[0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 1, 1, 0, 1],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
 [1, 0, 0, 0, 1, 0, 1, 0, 0, 1]]

Estamos ya en condiciones de resolver el problema planteado mediante un algoritmo genético. Este puede ser uno implementado _ad hoc_ por nosotros, o puede ser uno de los ya implementados en el paquete DEAP (véase http://deap.readthedocs.org/en/1.0.x/api/algo.html).

Por ejemplo, la función `eaSimple` del módulo `algorithms` implementa el siguiente algoritmo genético:
1. Evaluar los individuos de la población inicial.
2. Repetir para el número de generaciones especificado:
  1. Aplicar el procedimiento de selección para reemplazar a la población por completo.
  2. Para cada par de individuos $x_{i}$ y $x_{i+1}$, cruzarlos con la probabilidad especificada, siendo reemplazados por sus hijos en caso de que sí se crucen.
  3. Para cada individuo, mutarlo con la probabilidad especificada, siendo reemplazado por el nuevo individuo en caso de que sí mute.
  4. Evaluar los individuos de la nueva población.

__Nota de implementación__: el _fitness_ de los individuos que permanecen de una generación a la siguiente no se reevalúa. El resultado devuelto por el algoritmo es una tupla con la población final y un registro indicando para cada generación a cuántos individuos nuevos ha debido calcularse su _fitness_.

In [21]:
random.seed(12345)
pobInicial = cajaTarjetas.generaPobTarjetas()
pobFinal, registro = algorithms.eaSimple(pobInicial, cajaTarjetas, 
                                         cxpb = 0.5,  # Probabilidad de cruzamiento
                                         mutpb = 0.3,  # Probabilidad de mutación
                                         ngen = 20,  # Número de generaciones
                                         verbose = False)

for individuo in pobFinal:
    print(individuo, fenotipoIndTarjetas(individuo), cajaTarjetas.evaluate(individuo))

print(registro)

[0, 0, 0, 0, 1, 0, 1, 0, 0, 1] ([1, 2, 3, 4, 6, 8, 9], [5, 7, 10]) (13,)
[1, 0, 0, 0, 0, 1, 1, 0, 0, 1] ([2, 3, 4, 5, 8, 9], [1, 6, 7, 10]) (65,)
[0, 0, 0, 0, 1, 0, 1, 1, 0, 1] ([1, 2, 3, 4, 6, 9], [5, 7, 8, 10]) (2451,)
[0, 0, 0, 0, 1, 0, 1, 0, 0, 1] ([1, 2, 3, 4, 6, 8, 9], [5, 7, 10]) (13,)
[0, 0, 0, 0, 1, 0, 1, 0, 0, 1] ([1, 2, 3, 4, 6, 8, 9], [5, 7, 10]) (13,)
[0, 1, 0, 0, 1, 0, 1, 0, 0, 0] ([1, 3, 4, 6, 8, 9, 10], [2, 5, 7]) (295,)
[0, 0, 0, 0, 1, 0, 1, 0, 0, 1] ([1, 2, 3, 4, 6, 8, 9], [5, 7, 10]) (13,)
[0, 0, 0, 0, 1, 0, 1, 0, 0, 1] ([1, 2, 3, 4, 6, 8, 9], [5, 7, 10]) (13,)
[0, 0, 0, 0, 1, 0, 1, 0, 0, 1] ([1, 2, 3, 4, 6, 8, 9], [5, 7, 10]) (13,)
[1, 0, 0, 1, 1, 1, 1, 0, 0, 1] ([2, 3, 8, 9], [1, 4, 5, 6, 7, 10]) (8054,)
gen	nevals
0  	10    
1  	8     
2  	8     
3  	6     
4  	5     
5  	8     
6  	5     
7  	7     
8  	7     
9  	4     
10 	8     
11 	5     
12 	8     
13 	8     
14 	5     
15 	8     
16 	6     
17 	5     
18 	4     
19 	4     
20 	7     


Todas las funciones del paquete DEAP que implementan algoritmos genéticos admiten el cálculo de estadísticas y un _salón de la fama_ con los mejores individuos que hayan aparecido a lo largo de la evolución de la población.

In [22]:
# Estadísticas sobre el fitness de los individuos: mínimo, media y máximo
# Nota: usamos para ello las funciones correspondientes de numpy, porque los
# valores de fitness están guardados en tuplas
estadísticas = tools.Statistics(lambda ind: ind.fitness.values)
estadísticas.register("mínimo", numpy.min)
estadísticas.register("media", numpy.mean)
estadísticas.register("máximo", numpy.max)

# Salón de la fama para recopilar los tres mejores individuos de todas las generaciones
salón_fama = tools.HallOfFame(3)

In [23]:
random.seed(12345)
pobInicial = cajaTarjetas.generaPobTarjetas()
pobFinal, registro = algorithms.eaSimple(pobInicial, cajaTarjetas, 
                                         cxpb = 0.5,  # Probabilidad de cruzamiento
                                         mutpb = 0.3,  # Probabilidad de mutación
                                         ngen = 20,  # Número de generaciones
                                         stats = estadísticas,
                                         halloffame = salón_fama)

gen	nevals	mínimo	media 	máximo
0  	10    	4     	2052.9	9735  
1  	8     	14    	2264.8	18559 
2  	8     	13    	388.6 	2803  
3  	6     	13    	122   	1048  
4  	5     	13    	65.6  	346   
5  	8     	13    	255.2 	1749  
6  	5     	13    	6725.7	66861 
7  	7     	13    	216.8 	1750  
8  	7     	13    	41.1  	292   
9  	4     	13    	152.1 	1047  
10 	8     	13    	251.8 	1048  
11 	5     	13    	79.6  	346   
12 	8     	13    	292   	2802  
13 	8     	13    	81.3  	696   
14 	5     	13    	2844.7	24858 
15 	8     	13    	1750.7	10855 
16 	6     	13    	449.8 	2451  
17 	5     	13    	88.1  	696   
18 	4     	13    	13    	13    
19 	4     	13    	186.6 	1749  
20 	7     	13    	1094.3	8054  


In [24]:
print('Las tres mejores soluciones encontradas han sido:')
for individuo in salón_fama:
    print('Individuo: P1={1}, P2={2}; Fitness: {0}'.format(
        individuo.fitness.values[0], *fenotipoIndTarjetas(individuo)))

Las tres mejores soluciones encontradas han sido:
Individuo: P1=[2, 3, 4, 6, 7, 10], P2=[1, 5, 8, 9]; Fitness: 4.0
Individuo: P1=[1, 2, 3, 4, 6, 8, 9], P2=[5, 7, 10]; Fitness: 13.0
Individuo: P1=[2, 3, 4, 6, 8, 9], P2=[1, 5, 7, 10]; Fitness: 14.0


## Resumen

### Genes e individuos - cruce y mutación
- Hemos visto un ejemplo en el que usamos individuos con genes binarios
    - Mutación de un individuo (independiente para cada gen): intercambio de bit
    - Cruce de dos individuos: intercambio de valores (cruce en un punto y otras variantes)
- Los genes también pueden ser números naturales
    - Mutación de un individuo (independiente para cada gen): reemplazo del gen por un valor generado aleatoriamente dentro de un rango o siguiendo cierta distribución estadística
    - Cruce de dos individuos: intercambio de valores (cruce en un punto y otras variantes)
- ¿Y si nuestro individuo está compuesto por números naturales sin repetición? ¿Tienen validez los operadores anteriores?
    - Mutación de un individuo (varios genes participan): intercambio de genes (mutación por intercambio y otras variantes)
    - Cruce de dos individuos: intercambio de valores evitando repeticiones (cruce basado en orden y cruce basado en ciclos)
    
#### Operadores de cruce y mutación implementados en la librería DEAP:
A continuación se enumeran los operadores vistos en clase. En la documentación podemos consultarlos todos: https://deap.readthedocs.io/en/master/api/tools.html#operators

- Genes binarios y numéricos - mutación:
    - Intercambio de bit (solo binario): https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutFlipBit
    - Reemplazo por otro valor generado aleatoriamente: https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutUniformInt
    - Mutación por intercambio (permutación): https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutShuffleIndexes
- Genes binarios y numéricos - cruce:
    - Cruce en un punto: https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxOnePoint
    - Cruce en dos puntos:https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxTwoPoint
    - Cruce multipunto: https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxUniform
- Genes numéricos sin repetición (permutaciones):
    - cruce basado en orden: https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxOrdered
    
### Operadores de selección

- Selección por ruleta: 
    - Es el método de selección visto en clase
    - Selección aleatoria de individuos
    - La probabilidad para cada individuo de ser seleccionado, varía en función de su utilidad
    - **Solo válido para problemas de maximización**
- Selección por torneo:
    - Para seleccionar N individuos, se realizan N torneos
    - Un torneo consiste en seleccionar aleatoriamente K individuos y devolver aquel con mayor utilidad
    - **Válido tanto para problemas de maximización como de minimización**
    
#### Operadores selección implementados en la librería DEAP:

- Selección por ruleta: https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.selRoulette
- Selección por torneo:https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.selTournament

# El problema de la mochila

El _problema de la mochila_ (_knapsack problem_) es un problema de optimización combinatoria que aparece en procesos de decisión del mundo real en una amplia variedad de campos. El problema se puede enunciar de manera abstracta como sigue:
> Dado un conjunto de elementos, cada uno de ellos con un peso y un valor, determinar la cantidad de cada elemento a incluir en una colección de tal manera que el peso total sea menor o igual que un límite dado y el valor total sea lo mayor posible.

En esta práctica nos restringiremos al problema de la mochila 0-1, en el que cada elemento solo se puede incluir a lo sumo una vez en la colección. En concreto, consideraremos la instancia que consiste de 1000 elementos, con los pesos y valores recogidos en el fichero `instancia_knapsack.txt`, y con un peso límite de 4816. Para esta instancia la solución óptima tiene un valor de 27147.

Los pesos y los valores de los elementos de la instancia se pueden leer del fichero evaluando las siguientes expresiones:

In [25]:
PESOS = []
VALORES = []
with open('instancia_knapsack.txt', 'r') as fichero:
    for línea in fichero:
        peso, valor = tuple(línea.split())
        PESOS.append(int(peso))
        VALORES.append(int(valor))
        
print(PESOS[1:50])        
print(VALORES[1:50])        

[506, 416, 992, 649, 237, 457, 815, 446, 422, 791, 359, 667, 598, 7, 544, 334, 766, 994, 893, 633, 131, 428, 700, 617, 874, 720, 419, 794, 196, 997, 116, 908, 539, 707, 569, 537, 931, 726, 487, 772, 513, 81, 943, 58, 303, 764, 536, 724, 789]
[886, 814, 1151, 983, 629, 848, 1074, 839, 819, 1062, 762, 994, 950, 111, 914, 737, 1049, 1152, 1110, 973, 474, 824, 1013, 963, 1101, 1024, 816, 1063, 575, 1153, 447, 1117, 910, 1017, 931, 909, 1126, 1027, 871, 1052, 891, 375, 1131, 318, 705, 1048, 908, 1026, 1061]


__Ejercicio 1__: para resolver un problema de la mochila 0-1 mediante algoritmos genéticos definimos los parámetros necesarios como sigue:
* Los genes son 0 y 1.
* Los individuos son cromosomas de longitud la cantidad de elementos disponibles.
* Cada individuo representa la solución en la que el elemento $i$-ésimo se incluye en la colección si y solo si el gen $i$-ésimo es 1.
* La evaluación del fenotipo de cada individuo es la suma de los valores de los elementos incluidos en la colección, salvo que el peso supere el límite establecido, en cuyo caso es $-\infty$ (penalización por _pena de muerte_).

Se pide implementar esta representación en una caja de herramientas que permita realizar las pruebas experimentales que se piden en el ejercicio 5.

In [26]:
# import random
# from deap import base, creator, tools, algorithms
# import numpy
# import math
#
# <NOM_CajaHerramientas> = base.Toolbox()

# creator.create('<NOM_CriterioEvaluacion>', base.Fitness, weights = <tuplaPesos>)
# creator.create('<NOM_EstructuraIndividuo>', list, fitness = creator.NOM_CriterioEvaluacion)

# <NOM_CajaHerramientas>.register('<NOM_CrearGen>', ...)

# <NOM_CajaHerramientas>.register('<NOM_CrearIndividuo>', tools.initRepeat,
#                                 container = creator.<NOM_EstructuraIndividuo>, 
#                                 func = <NOM_CajaHerramientas>.<NOM_CrearGen>, 
#                                 n = <N_tamIndividuo>)
# <NOM_CajaHerramientas>.register('<NOM_CrearPoblacion>', tools.initRepeat,
#                                 container = list, 
#                                 func = <NOM_CajaHerramientas>.<NOM_CrearIndividuo>, 
#                                 n = <N_tamPoblacion>)
#
# def <Nom_Fitness>(individuo):
#   ...
#
# <NOM_CajaHerramientas>.register('evaluate', <Nom_Fitness>)
#
# <NOM_CajaHerramientas>.register('mate', ...)
# <NOM_CajaHerramientas>.register('mutate', ...)
# <NOM_CajaHerramientas>.register('select', ...)
#
# <NOM_Estadisticas> = tools.Statistics(lambda ind: ind.fitness.values)
# <NOM_Estadisticas>.register("mínimo", numpy.min)
# <NOM_Estadisticas>.register("media", numpy.mean)
# <NOM_Estadisticas>.register("máximo", numpy.max)
#
# <NOM_SalonFama> = tools.HallOfFame(<N_mejoresSeleccionados>)
#
# random.seed(12345)
# población, registro = algorithms.eaSimple(<NOM_CajaHerramientas>.<NOM_CrearPoblacion>(),
#                                           <NOM_CajaHerramientas>,
#                                           cxpb = 0.5, # Probabilidad de que dos individuos contiguos se crucen
#                                           mutpb = 0.3, # Probabilidad de que un individuo mute
#                                           ngen = 20, # Número de generaciones
#                                           stats = <NOM_Estadisticas>,
#                                           halloffame = <NOM_SalonFama>)
#
# print('Las tres mejores soluciones encontradas han sido:')
# for individuo in <NOM_SalonFama>:
#     print('Individuo con fitness: {0}'.format(individuo.fitness.values[0]))

In [27]:
# Incluimos aquí valores para algunos parámatros
T_IND = 1000 # Tamaño de los individuos
T_POB = 1000  # Tamaño de la población

In [28]:
# <NOM_CajaHerramientas> <-> CajaMochila
CajaMochila = base.Toolbox()

# <NOM_CriterioEvaluacion> <-> critMax
creator.create('critMax', base.Fitness, weights = (1,))

# <NOM_EstructuraIndividuo> <-> indMochila01
creator.create('indMochila01', list, fitness = creator.critMax)

# <NOM_CrearGen> <-> gen01
CajaMochila.register('gen01', random.randint, 0, 1)

# <NOM_CrearIndividuo> <-> crearIndMochila01
CajaMochila.register('crearIndMochila01', tools.initRepeat,
                     container = creator.indMochila01, 
                     func = CajaMochila.gen01, 
                     n = T_IND)

In [29]:
for _ in range(10): print(CajaMochila.crearIndMochila01()[1:10])

[1, 1, 1, 1, 0, 1, 1, 1, 0]
[1, 1, 1, 0, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 0, 1, 0, 0, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 0]
[1, 0, 1, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 1]
[1, 1, 0, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 0, 1, 1, 1, 1, 0, 1]
[0, 0, 1, 1, 1, 0, 0, 0, 1]


In [30]:
CajaMochila.register('generaPobMochila01', tools.initRepeat,
                     container = list, 
                     func = CajaMochila.crearIndMochila01, 
                     n = T_POB)

In [31]:
pob = CajaMochila.generaPobMochila01()
pob[499]

[1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [32]:
def fen1(individuo): #decodifica
    mochila=[]
    peso_ac = 0
    for i in range individuo:
        peso_ac+=peso[i]
        if peso_ac>4816:
            return mochila
        mochila.append(i)
    
    return (i for i in range(T_IND) if individuo[i])

def evind1(individuo):
    fen = fen1(individuo)
    sumaP = sum (PESOS[i] for i in fen)
    if (sumaP<=4816):
        return (sum (VALORES[i] for i in fen),)
    else:
        return (float("-inf"),)

# Definiciones vistas en clase:
# Primera

def FenMochilaV1(individuo):
    mochila = []
    for ind in range(1000):
        if (individuo[ind] == 1):
            mochila.append(ind)
    return mochila

def evalMochilaV1(individuo):
    mochila = FenMochilaV1(individuo)
    pesoM = 0
    for obj in mochila:
        pesoM += PESOS[obj]
    if (pesoM <= 4816):
        valorM = 0
        for obj in mochila:
            valorM += VALORES[obj]
        return (valorM, )
    else:
        return (float("-inf"),)
    
# Segunda 
def valorMochila01_version1(individuo):
    peso = 0
    for ind in range(T_IND):
        peso += individuo[ind]*PESOS[ind]
    if (peso > 4816):
        valor = float("-inf")
    else:
        valor = 0
        for ind in range(T_IND):
            valor += individuo[ind]*VALORES[ind]
    return (valor,)
CajaMochila.register('evaluate', evind1)

In [33]:
prueba = CajaMochila.crearIndMochila01()
CajaMochila.evaluate(prueba)

(-inf,)

In [34]:
#
CajaMochila.register('mate', tools.cxOnePoint)
CajaMochila.register('mutate', tools.mutFlipBit, indpb=0.1)
CajaMochila.register('select',  tools.selTournament, tournsize=3)
#

In [35]:
# <NOM_Estadisticas> = tools.Statistics(lambda ind: ind.fitness.values)
# <NOM_Estadisticas>.register("mínimo", numpy.min)
# <NOM_Estadisticas>.register("media", numpy.mean)
# <NOM_Estadisticas>.register("máximo", numpy.max)
#
# <NOM_SalonFama> = tools.HallOfFame(<N_mejoresSeleccionados>)
#
# random.seed(12345)
población, registro = algorithms.eaSimple(CajaMochila.generaPobMochila01(),
                                          CajaMochila,
                                          cxpb = 0.5, # Probabilidad de que dos individuos contiguos se crucen
                                          mutpb = 0.3, # Probabilidad de que un individuo mute
                                          ngen = 20, # Número de generaciones
                                          stats = estadísticas,
                                          halloffame = salón_fama)
#
print('Las tres mejores soluciones encontradas han sido:')
for individuo in salón_fama:
    print('Individuo con fitness: {0}'.format(individuo.fitness.values[0]))

gen	nevals	mínimo	media	máximo
0  	1000  	-inf  	-inf 	-inf  
1  	660   	-inf  	-inf 	-inf  
2  	629   	-inf  	-inf 	-inf  
3  	647   	-inf  	-inf 	-inf  
4  	649   	-inf  	-inf 	-inf  
5  	647   	-inf  	-inf 	-inf  
6  	636   	-inf  	-inf 	-inf  
7  	633   	-inf  	-inf 	-inf  
8  	626   	-inf  	-inf 	-inf  
9  	678   	-inf  	-inf 	-inf  
10 	637   	-inf  	-inf 	-inf  
11 	623   	-inf  	-inf 	-inf  
12 	652   	-inf  	-inf 	-inf  
13 	655   	-inf  	-inf 	-inf  
14 	635   	-inf  	-inf 	-inf  
15 	630   	-inf  	-inf 	-inf  
16 	657   	-inf  	-inf 	-inf  
17 	674   	-inf  	-inf 	-inf  
18 	625   	-inf  	-inf 	-inf  
19 	650   	-inf  	-inf 	-inf  
20 	701   	-inf  	-inf 	-inf  
Las tres mejores soluciones encontradas han sido:
Individuo con fitness: 4.0
Individuo con fitness: 13.0
Individuo con fitness: 14.0


__Ejercicio 2__: para resolver un problema de la mochila 0-1 mediante algoritmos genéticos definimos los parámetros necesarios como sigue:
* Los genes son 0 y 1.
* Los individuos son cromosomas de longitud la cantidad de elementos disponibles.
* Cada individuo representa la solución en la que el elemento $i$-ésimo se incluye en la colección si y solo si el gen $i$-ésimo es 1.
* La evaluación del fenotipo de cada individuo es la suma de los valores de los elementos incluidos en la colección, salvo que el peso supere el límite establecido, en cuyo caso se le resta como penalización la cantidad en la que supera ese límite, multiplicada opcionalmente por un $w$ prefijado de antemano.

Se pide implementar esta representación en una caja de herramientas que permita realizar las pruebas experimentales que se piden en el ejercicio 5.

In [36]:
def fen2(individuo):
    return (i for i in range(T_IND) if individuo[i])
def evind2(individuo,w):
    peso_acumulado = 0
    valor_acumulado = 0
    for i in fen2(individuo):
        peso_acumulado += PESOS[i]
        valor_acumulado += VALORES[i]
    if peso_acumulado > 4816:
        valor_acumulado -= w * (peso_acumulado - 4816)
    return (valor_acumulado,)

## Definición vista en clase:

def valorMochila01_version2(individuo, k):
    peso = 0
    valor = 0
    for ind in range(T_IND):
        peso += individuo[ind]*PESOS[ind]
        valor += individuo[ind]*VALORES[ind]
    if (peso > 4816):
        valor -= k * (peso - 4816)
        
    return (valor,)

CajaMochila.register('evaluate', evind2, w = 10)

población, registro = algorithms.eaSimple(CajaMochila.generaPobMochila01(),
                                          CajaMochila,
                                          cxpb = 0.5, # Probabilidad de que dos individuos contiguos se crucen
                                          mutpb = 0.3, # Probabilidad de que un individuo mute
                                          ngen = 20, # Número de generaciones
                                          stats = estadísticas,
                                          halloffame = salón_fama)
#
print('Las tres mejores soluciones encontradas han sido:')
for individuo in salón_fama:
    print('Individuo con fitness: {0}'.format(individuo.fitness.values[0]))

gen	nevals	mínimo      	media       	máximo      
0  	1000  	-2.18828e+06	-1.97506e+06	-1.74009e+06
1  	646   	-2.09386e+06	-1.91542e+06	-1.7106e+06 
2  	651   	-2.04859e+06	-1.87137e+06	-1.63233e+06
3  	618   	-2.01972e+06	-1.83127e+06	-1.6202e+06 
4  	625   	-1.98613e+06	-1.79381e+06	-1.63233e+06
5  	636   	-1.94375e+06	-1.76083e+06	-1.55786e+06
6  	632   	-1.92646e+06	-1.73084e+06	-1.55524e+06
7  	644   	-1.89393e+06	-1.69965e+06	-1.53049e+06
8  	621   	-1.85909e+06	-1.67003e+06	-1.50352e+06
9  	638   	-1.9154e+06 	-1.64372e+06	-1.47702e+06
10 	617   	-1.87648e+06	-1.61944e+06	-1.46237e+06
11 	632   	-1.8539e+06 	-1.59592e+06	-1.43353e+06
12 	668   	-1.77815e+06	-1.57126e+06	-1.39006e+06
13 	617   	-1.7899e+06 	-1.54636e+06	-1.38329e+06
14 	666   	-1.732e+06  	-1.52736e+06	-1.36174e+06
15 	651   	-1.74198e+06	-1.50194e+06	-1.35638e+06
16 	658   	-1.6912e+06 	-1.47757e+06	-1.3392e+06 
17 	659   	-1.69065e+06	-1.45704e+06	-1.2902e+06 
18 	665   	-1.6509e+06 	-1.437e+06  	-1.27618e+06


__Ejercicio 3__: para resolver un problema de la mochila 0-1 mediante algoritmos genéticos definimos los parámetros necesarios como sigue:
* Los genes son 0 y 1.
* Los individuos son cromosomas de longitud la cantidad de elementos disponibles.
* Cada individuo representa la solución en la que se van considerando en orden los elementos, de tal forma que el elemento $i$-ésimo se incluye en la colección si y solo si el gen $i$-esimo es 1 y no se supera el peso límite.
* La evaluación del fenotipo de cada individuo es la suma de los valores de los elementos incluidos en la colección.

Se pide implementar esta representación en una caja de herramientas que permita realizar las pruebas experimentales que se piden en el ejercicio 5.

__Ejercicio 4__: para resolver un problema de la mochila 0-1 mediante algoritmos genéticos definimos los parámetros necesarios como sigue:
* Los genes son los elementos disponibles.
* Los individuos son permutaciones de los genes.
* Cada individuo representa la solución en la que se van considerando en orden los genes, de tal forma que el elemento representado por el gen se incluye en la colección si no se supera el peso límite.
* La evaluación del fenotipo de cada individuo es la suma de los valores de los elementos incluidos en la colección.

Se pide implementar esta representación en una caja de herramientas que permita realizar las pruebas experimentales que se piden en el ejercicio 5.

__Ejercicio 5__: realizar pruebas experimentales en las que se intente resolver la instancia del problema de la mochila 0-1 proporcionada mediante los algoritmos genéticos implementados en el paquete DEAP, usando las representaciones de los ejercicios 1 a 4, y considerando distintos valores para los parámetros de los algoritmos.

Hola


¿Con cuál de las cuatro representaciones has obtenido mejores resultados? ¿Puedes razonar alguna explicación para ello?